### So, for the last few screen shots, you must have heard how bad the website is --- so now scrape away! For the Bot!

In [2]:
import pandas as pd
import re

First, I have downloaded the "root" html page, to get all the ids of organizations. 

In [8]:
from bs4 import BeautifulSoup
soup = BeautifulSoup('./www.partners4employment.ca/student-alumni.htm', 'html.parser')

/home/penpen/anaconda3/lib/python3.5/site-packages/bs4/__init__.py:219: UserWarning: "b'./www.partners4employment.ca/student-alumni.htm'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


the reason I didn't use soup for this step is because... it's aon over kill. 

In [9]:
reg_str = 'registrationId : '
len_reg_str = len(reg_str)
ids = []
with open('./www.partners4employment.ca/student-alumni/current-participating-organizations.htm') as f:
    content = f.readlines()
    for l in content: 
        m = re.search('registrationId : ([0-9]+)', l)
        if (m is not None):
            ids = ids + [m.group(0)[len_reg_str:]]

print("number of companies at jobfair: ", len(ids))

number of companies at jobfair:  194


## Sending Requests to get stuff we need

In [4]:
import requests

we're using some test data to see how the page should be parsed. 

In [12]:
endpoint = "https://www.partners4employment.ca/student-alumni/current-participating-organizations.htm"
data = {'action':'displayRegInfo', 
       'registrationId': '3279'} 
r = requests.post(url = endpoint, data = data)
job_soup = BeautifulSoup(r.text, "lxml")

In [15]:
name = job_soup.find("h1").text.strip()
print("name of company: ", name)
profile = job_soup.find_all("div", class_="controls-text")
counter = 0
for p in profile: 
    print("#", counter)
    print(p.text.strip())
    print("-----------------------------------")
    counter = counter + 1

name of company:  Think Research
# 0
78
-----------------------------------
# 1
Think Research is changing the way healthcare's delivered - no, really! - and not in the Buckley's "tastes awful but it works" kind of way. We are building software to give clinicians the information they need to treat patients better and faster. Why Us? It's not every day you get to change the way your friends and family are cared for. Our culture is one of the things we're most proud of--our fun, freindly and talented team will become your second family! Did we mention we're located in the core of Downtown Toronto?
-----------------------------------
# 2
Learn more at www.thinkresearch.com/ca/company/careers/engineering/. 


Engineering Co-op Students for the Summer Waterloo Co-op Term ( May 14th start): 4 month co-ops for the summer
Graduates/Alumni: Ruby Developers, Software Developers (full time)
-----------------------------------
# 3
Full-time|Co-op/Internship|Contract
-------------------------------

Okay, so we have description, address, name, phone number, etc. 
TL;DR: names and info have to be stripped seperately, but they're there. 


## Actual stripping script: 

In [16]:
company_df = pd.DataFrame(columns=[
    'id', 
    'name', 
    'booth', 
    'profile', 
    'positions', 
    'employment types', 
    'location', 
    'linkedin',
    'twitter',
    'facebook', 
    'contact'
])

for currid in ids: 
    endpoint = "https://www.partners4employment.ca/student-alumni/current-participating-organizations.htm"
    data = {'action':'displayRegInfo', 
           'registrationId': currid}
    r = requests.post(url = endpoint, data = data)
    job_soup = BeautifulSoup(r.text,  "lxml")
    
    name = job_soup.find("h1").text.strip()
    details = job_soup.find_all("div", class_="controls-text")
    boothnum = details[0].text.strip()
    profile = details[1].text.strip()
    positions = details[2].text.strip().split(' - ')
    employmentTypes = details[3].text.strip().split('/')
    location = details[4].text.strip().split('|')
    linkedin = details[5].text.strip()
    twitter = details[6].text.strip()
    facebook = details[7].text.strip()
    contact = details[8].text.strip()
    
    company_df = company_df.append({
        'id': currid,
        'name': name, 
        'booth': boothnum, 
        'profile':profile, 
        'positions':positions, 
        'employment types':employmentTypes, 
        'location': location, 
        'linkedin': linkedin,
        'twitter': twitter,
        'facebook':facebook, 
        'contact':contact
    }, ignore_index=True)
    print("added ", name, " currlen: ", company_df.size)

added  Accedo  currlen:  11
added  Adastra Corporation  currlen:  22
added  Aecon  currlen:  33
added  Aerotek  currlen:  44
added  African Lion Safari  currlen:  55
added  Agriculture & Agri-Food Canada  currlen:  66
added  Andiamo  currlen:  77
added  Arcane  currlen:  88
added  Arctic Glacier Canada  currlen:  99
added  Arvato  currlen:  110
added  Auvik Networks  currlen:  121
added  Aviva Canada - Healthcare Claims  currlen:  132
added  B&R Industrial Automation  currlen:  143
added  BASF Corporation  currlen:  154
added  BBM Canada  currlen:  165
added  Big Viking Games  currlen:  176
added  BlackBerry Limited  currlen:  187
added  Bonanza Gardens  currlen:  198
added  Brock Solutions  currlen:  209
added  BSM Technologies  currlen:  220
added  BWXT Canada Ltd.  currlen:  231
added  Camp Couchiching  currlen:  242
added  Camp Kennebec  currlen:  253
added  Camp Kodiak  currlen:  264
added  Camp Trillium  currlen:  275
added  Canada Revenue Agency  currlen:  286
added  Canadian Br